In [1]:
import os 
import time 
import random
import numpy as np 

from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

from tensorflow.keras.callbacks import TensorBoard

from data_prepare.save_data import save
from data_prepare.data_splitter import get_train_and_test_data as gttd


In [2]:
level = 'BigFans'
save(level, delete=False, debug=True)
os.chdir('D:/Documents/Python/fallguy_bot/DATA/LEVELS/'+ level)

Already saved


In [3]:
X = np.load(open('X.npy','rb'))
y = np.load(open('y.npy','rb'))


In [4]:
X_train, y_train, X_test, y_test = gttd(X,y)

6879
creating X_test
creating y test
creating X_train
creating y_train
try to return


In [6]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(6192, 240, 427, 1)
(6192, 7)
(687, 240, 427, 1)
(687, 7)


In [7]:
del X, y
X_train = X_train/255.0
X_test = X_test/255.0

y_train = np.asarray(y_train).astype('float32')
y_test = np.asarray(y_test).astype('float32')


In [17]:
key_pos = 0

press_number = 0 
what_label = y_train
for label in what_label:
    for index, num in enumerate(label):
        if num == 1 and index==key_pos:
            press_number += 1
            
print(f'In {len(what_label)} categories. You pressed key with position {key_pos},\
    {press_number} times')

print(f'its have {press_number*100/len(what_label)} percent')

print(y_train[0])

In 6192 categories. You pressed key with position 0,    3756 times
its have 60.65891472868217 percent
[1. 1. 0. 0. 0. 0. 0.]


In [9]:
X_train.shape[1:]

(240, 427, 1)

In [14]:
#My fucked up model 
# model = Sequential()

# model.add(Conv2D((64), (3,3), input_shape = X_train.shape[1:]))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))

# model.add(Conv2D((32), (2,2)))
# model.add(Activation("relu"))
# model.add(MaxPooling2D(pool_size=(2,2)))


# model.add(Flatten())
# model.add(Dense(8))
# model.add(Activation('relu'))


# model.add(Dense(7))
# model.add(Activation('sigmoid'))

# model.compile(
#     loss="binary_crossentropy",
#     optimizer='adam',
#     metrics=['accuracy'],
# )
# model.summary()

model = Sequential()

# 2 sets of CRP (Convolution, RELU, Pooling)
model.add(Conv2D(20, (5, 5), padding="same",
	input_shape=X_train.shape[1:], kernel_regularizer=l2(0.)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

model.add(Conv2D(50, (5, 5), padding="same",
	kernel_regularizer=l2(0.)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

# Fully connected layers (w/ RELU)
model.add(Flatten())
model.add(Dense(500, kernel_regularizer=l2(0.)))
model.add(Activation("relu"))

# Softmax (for classification)
model.add(Dense(len(y_train[0]), kernel_regularizer=l2(0.)))
model.add(Activation("softmax"))

model.compile(
    loss="binary_crossentropy",
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 240, 427, 20)      520       
_________________________________________________________________
activation_4 (Activation)    (None, 240, 427, 20)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 120, 213, 20)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 120, 213, 50)      25050     
_________________________________________________________________
activation_5 (Activation)    (None, 120, 213, 50)      0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 60, 106, 50)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 318000)           

In [15]:
NAME = 'CNN-{}'.format(int(time.time()))
tensorboard = TensorBoard(log_dir='logs/{}'.format(NAME))

In [16]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_test, y_test), callbacks=[tensorboard])
model.save(f'cnn_{level}.model')

Epoch 1/10
97/97 [==============================] - 404s 4s/step - loss: 1.1342 - accuracy: 0.5148 - val_loss: 0.2750 - val_accuracy: 0.6041
Epoch 2/10
97/97 [==============================] - 394s 4s/step - loss: 0.1723 - accuracy: 0.6491 - val_loss: 0.2091 - val_accuracy: 0.6405
Epoch 3/10
97/97 [==============================] - 385s 4s/step - loss: 0.0534 - accuracy: 0.6785 - val_loss: 0.2280 - val_accuracy: 0.6114
Epoch 4/10
97/97 [==============================] - 390s 4s/step - loss: 0.0239 - accuracy: 0.6933 - val_loss: 0.2682 - val_accuracy: 0.6565
Epoch 5/10
97/97 [==============================] - 389s 4s/step - loss: 0.0149 - accuracy: 0.7154 - val_loss: 0.2909 - val_accuracy: 0.6172
Epoch 6/10
97/97 [==============================] - 404s 4s/step - loss: 0.0081 - accuracy: 0.7040 - val_loss: 0.3207 - val_accuracy: 0.6463
Epoch 7/10
97/97 [==============================] - 394s 4s/step - loss: 0.0055 - accuracy: 0.7209 - val_loss: 0.3179 - val_accuracy: 0.5328
Epoch 8/10
97